In [1]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
data_path = "../data/samples/sample.csv"
sample_df = pd.read_csv(data_path)
print(sample_df.shape)
sample_df.head()

(179940, 21)


/tmp/ipykernel_952/1509540579.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  sample_df = pd.read_csv(data_path)


,artist_followers,genres,album_total_tracks,artist_popularity,explicit,tempo,chart,album_release_date,energy,key,...,available_markets,mode,time_signature,speechiness,danceability,valence,acousticness,liveness,instrumentalness,loudness
0,1288.0,[],1.0,39.0,False,160.044,0.0,-9223372036854775808,0.345,5.0,...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",0.0,4.0,0.0458,0.513,0.1560,0.767000,0.1070,0.825,-8.166
1,7871.0,['progressive alternative'],11.0,18.0,False,120.010,0.0,-9223372036854775808,0.845,10.0,...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",0.0,3.0,0.0591,0.380,0.2950,0.000017,0.2730,0.727,-8.058
2,14059.0,['pet calming'],30.0,60.0,False,73.000,0.0,-9223372036854775808,0.102,5.0,...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",0.0,1.0,0.0429,0.210,0.0382,0.955000,0.1090,0.786,-17.755
3,2471.0,['piano cover'],10.0,31.0,False,69.612,0.0,-9223372036854775808,0.014,9.0,...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1.0,3.0,0.0492,0.436,0.0886,0.993000,0.0913,0.901,-30.246
4,1472617.0,"['filmi', 'modern bollywood']",7.0,50.0,False,150.756,0.0,-9223372036854775808,0.501,7.0,...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",0.0,3.0,0.0329,0.476,0.6600,0.665000,0.1640,0.000,-9.491


# Start with genres

In [3]:
genres_df = sample_df['genres'].fillna(value="[]").apply(eval).to_frame()
genres_df

,genres
0,[]
1,[progressive alternative]
2,[pet calming]
3,[piano cover]
4,"[filmi, modern bollywood]"
...,...
179935,"[deep house, float house]"
179936,[]
179937,[melodic techno]
179938,[zouk riddim]


In [4]:
mlb = MultiLabelBinarizer()
genres_categories = pd.DataFrame(mlb.fit_transform(genres_df["genres"]), columns=mlb.classes_, index=genres_df.index)
genres_categories.head()

,2-step,21st century classical,432hz,48g,528hz,5th wave emo,8-bit,8d,a cappella,aarhus indie,...,zim urban groove,zimdancehall,zither,zolo,zouglou,zouk,zouk riddim,zurich indie,zxc,zydeco
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
from scipy.sparse import csr_matrix

genres_counts = genres_categories.sum()
genres_counts.sort_values(ascending=False, inplace=True)
print(genres_counts)

sparse_genres_categories = csr_matrix(genres_categories.values)

sparse_genres_pairs_counts = sparse_genres_categories.T * sparse_genres_categories

genres_pairs_counts = pd.DataFrame(sparse_genres_pairs_counts.toarray(), index=genres_counts.index, columns=genres_counts.index)

# Zero out the diagonal
genres_pairs_counts.values[[range(genres_pairs_counts.shape[0])]*2] = 0

genres_pairs_counts

classical                 7126
rock                      5180
pop                       3183
classic rock              2920
rap                       2287
                          ... 
panamanian rock              1
bikutsi                      1
persian sad rap              1
perth hip hop                1
musica infantil catala       1
Length: 5305, dtype: int64


,classical,rock,pop,classic rock,rap,album rock,hard rock,soft rock,singer-songwriter,edm,...,orkiestra symfoniczna,ostschlager,bisrock,bisaya indie,palestinian traditional,panamanian rock,bikutsi,persian sad rap,perth hip hop,musica infantil catala
classical,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
rock,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pop,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
classic rock,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
rap,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
panamanian rock,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bikutsi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
persian sad rap,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
perth hip hop,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
quantiles_init = genres_counts.quantile([x / 20 for x in range(1, 21)]).to_frame().astype(int)
quantiles_init["Cumulative"] = genres_counts[::-1].cumsum().quantile([x / 20 for x in range(21)]).astype(int)
quantiles_init

,0,Cumulative
0.05,1,266
0.10,1,531
0.15,2,1049
0.20,3,1721
0.25,4,2642
0.30,5,3862
0.35,7,5454
0.40,9,7501
0.45,11,10105
0.50,14,13392


In [7]:
from joblib import Parallel, delayed
import pandas as pd
import re

features = sorted(genres_categories.columns, key=len)

# Function to check if one string is a substring of another with delimiters
def check_substring(feature1, feature2):
    pattern1 = r'\b' + re.escape(feature1) + r'\b'
    
    return re.search(pattern1, feature2) is not None


# Function to process a pair of features
def process_pair(i, j, features):
    return (features[i], features[j], check_substring(features[i], features[j]))

# List to store results
results = []

# Use Parallel and delayed to parallelize the computation
n_jobs = -1  # Use all available cores
results = Parallel(n_jobs=n_jobs, backend="multiprocessing")(
    delayed(process_pair)(i, j, features) for i in range(len(features)) for j in range(i + 1, len(features))
)

# Convert results to DataFrame for better readability
results_df = pd.DataFrame(results, columns=['Feature1', 'Feature2', 'IsSubstring'])

# Filter only the pairs where one is a substring of the other
substring_pairs = results_df[results_df['IsSubstring']]

print(substring_pairs)

                          Feature1                         Feature2  \
27939                          bgm                      chinese bgm   
42953                          ccm                         deep ccm   
43526                          ccm                       german ccm   
44231                          ccm                     canadian ccm   
44628                          ccm                    brazilian ccm   
...                            ...                              ...   
14028904   experimental electronic    dutch experimental electronic   
14028908   experimental electronic    irish experimental electronic   
14028933   experimental electronic   polish experimental electronic   
14028947   experimental electronic  russian experimental electronic   
14041102  american classical piano   latin american classical piano   

          IsSubstring  
27939            True  
42953            True  
43526            True  
44231            True  
44628            True  
...

In [8]:
composite_genres = set(substring_pairs['Feature2'])
atomic_genres = set(features) - composite_genres

In [9]:
composite_genres = pd.DataFrame([composite_genres, genres_counts[composite_genres]]).T
composite_genres.columns = ['Genre', 'Count']
composite_genres.set_index('Genre', inplace=True)
composite_genres.sort_values('Count', ascending=False, inplace=True)
composite_genres

/tmp/ipykernel_952/2917886796.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  composite_genres = pd.DataFrame([composite_genres, genres_counts[composite_genres]]).T


,Count
Genre,
classic rock,2920
album rock,2188
hard rock,2098
soft rock,2051
alternative metal,1963
...,...
reggae boliviano,1
greek jazz,1
gabonese pop,1


In [10]:
atomic_genres = pd.DataFrame([atomic_genres, genres_counts[atomic_genres]]).T
atomic_genres.columns = ['Genre', 'Count']
atomic_genres.set_index('Genre', inplace=True)
atomic_genres.sort_values('Count', ascending=False, inplace=True)
atomic_genres

/tmp/ipykernel_952/3904051305.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  atomic_genres = pd.DataFrame([atomic_genres, genres_counts[atomic_genres]]).T


,Count
Genre,
classical,7126
rock,5180
pop,3183
rap,2287
singer-songwriter,2006
...,...
trio cubano,1
wuhan indie,1
thai indie,1


In [11]:
# Get quantiles for the composite genres
quantiles_composite = composite_genres['Count'].quantile([x /100 for x in range(0, 101, 5)]).to_frame().astype(int)
quantiles_composite["Cumulative"] = composite_genres["Count"][::-1].cumsum().quantile([x /100 for x in range(0, 101, 5)]).astype(int)
quantiles_composite

,Count,Cumulative
0.00,1,1
0.05,1,161
0.10,2,327
0.15,2,648
0.20,3,1059
0.25,4,1624
0.30,5,2373
0.35,7,3367
0.40,9,4664
0.45,11,6295


In [12]:
# Get quantiles for the atomic genres
quantiles_atomic = atomic_genres['Count'].quantile([x /100 for x in range(0, 101, 5)]).to_frame().astype(int)
quantiles_atomic["Cumulative"] = atomic_genres["Count"][::-1].cumsum().quantile([x /100 for x in range(0, 101, 5)]).astype(int)
quantiles_atomic

,Count,Cumulative
0.00,1,1
0.05,1,105
0.10,1,210
0.15,2,402
0.20,3,664
0.25,4,1021
0.30,5,1491
0.35,7,2091
0.40,8,2860
0.45,10,3817


# Genres transformation pipeline
- Multilabel binarization
- Genres decomposition
- Composite genres drop
- "Others" binary feature. It will hold the unknown genres and genres with less than 20 entries in sample

In [13]:
for composite_g in composite_genres.index:
    components = substring_pairs[substring_pairs["Feature2"] == composite_g]["Feature1"].values
    genres_categories.loc[genres_categories[composite_g] == 1, components] = 1

transformed_genres = genres_categories.drop(columns=composite_genres.index)
transformed_genres.head()

,2-step,432hz,48g,528hz,8-bit,8d,a cappella,aarhus indie,abstract,abstractro,...,zillertal,zim urban groove,zimdancehall,zither,zolo,zouglou,zouk,zurich indie,zxc,zydeco
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
transformed_counts = transformed_genres.sum()
transformed_counts.sort_values(ascending=False, inplace=True)
transformed_counts

pop                      27393
rock                     20579
classical                 9708
jazz                      7969
hip hop                   7802
                         ...  
burkinabe traditional        1
lithuanian indie             1
starogradska                 1
london on indie              1
croatian indie               1
Length: 2092, dtype: int64

In [15]:
quantiles_transformed = transformed_counts.quantile([x / 20 for x in range(1, 21)]).to_frame().astype(int)
quantiles_transformed["Cumulative"] = transformed_counts[::-1].cumsum().quantile([x / 20 for x in range(21)]).astype(int)
quantiles_transformed

,0,Cumulative
0.05,1,105
0.10,1,210
0.15,2,408
0.20,3,676
0.25,4,1042
0.30,5,1524
0.35,7,2153
0.40,8,2935
0.45,11,3930
0.50,14,5199


In [16]:
genre_threshold = 20
transformed_genres["Others"] = transformed_genres[transformed_counts[transformed_counts <= genre_threshold].index].any(axis=1).astype(int)
transformed_genres = transformed_genres.drop(columns=transformed_counts[transformed_counts <= genre_threshold].index)
transformed_genres

,8-bit,8d,a cappella,abstract,accordeon,accordion,acoustic chill,acoustic cover,acoustic guitar cover,adoracao,...,xtra raw,yaoi,ye ye,yodeling,zillertal,zim urban groove,zolo,zouk,zydeco,Others
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179935,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179936,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179937,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179938,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [17]:
final_counts = transformed_genres.sum()
final_counts.sort_values(ascending=False, inplace=True)
print(final_counts)

pop                          27393
rock                         20579
classical                     9708
jazz                          7969
hip hop                       7802
                             ...  
south african alternative       21
arrochadeira                    21
russian dance                   21
corrosion                       21
eurobeat                        21
Length: 884, dtype: int64


# Available markets transformation
- Multilable binarization
- Done

In [18]:
av_markets_df = sample_df['available_markets'].fillna(value="[]").apply(eval).to_frame()
av_markets_df

,available_markets
0,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C..."
1,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C..."
2,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C..."
3,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C..."
4,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C..."
...,...
179935,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C..."
179936,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C..."
179937,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C..."
179938,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C..."


In [19]:
mlb = MultiLabelBinarizer()
transformed_av_markets = pd.DataFrame(mlb.fit_transform(av_markets_df["available_markets"]), columns=mlb.classes_, index=av_markets_df.index)
transformed_av_markets.head()

,AD,AE,AG,AL,AM,AO,AR,AT,AU,AZ,...,UZ,VC,VE,VN,VU,WS,XK,ZA,ZM,ZW
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [20]:
av_markets_counts = transformed_av_markets.sum()
av_markets_counts.sort_values(ascending=False, inplace=True)
print(av_markets_counts)

CW    129884
PT    129264
NO    129242
NL    129233
FI    129207
       ...  
LY    124036
RS    123661
XK    123661
BY    118786
PR    105535
Length: 185, dtype: int64


# Key, time_signature transformation
- one hot encode
- done

In [21]:
key_time_df = sample_df[['key', 'time_signature']]
key_time_df

,key,time_signature
0,5.0,4.0
1,10.0,3.0
2,5.0,1.0
3,9.0,3.0
4,7.0,3.0
...,...,...
179935,2.0,4.0
179936,9.0,4.0
179937,11.0,4.0
179938,1.0,4.0


In [22]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)
key_time_encoded = pd.DataFrame(ohe.fit_transform(key_time_df), columns=ohe.get_feature_names_out(), index=key_time_df.index)
key_time_encoded

,key_0.0,key_1.0,key_2.0,key_3.0,key_4.0,key_5.0,key_6.0,key_7.0,key_8.0,key_9.0,key_10.0,key_11.0,time_signature_0.0,time_signature_1.0,time_signature_3.0,time_signature_4.0,time_signature_5.0
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179935,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
179936,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
179937,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
179938,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# Continuous data transformation

In [23]:
normal_features = [
    "artist_followers",
    "album_total_tracks",
    "artist_popularity",
    "tempo",
    "speechiness",
    "danceability",
    "liveness",
    "loudness",
]

uniform_features = [
    "energy",
    "valence",
    "acousticness",
    "instrumentalness",
]

In [24]:
normal_features_df = sample_df[normal_features]
normal_features_df

,artist_followers,album_total_tracks,artist_popularity,tempo,speechiness,danceability,liveness,loudness
0,1288.0,1.0,39.0,160.044,0.0458,0.513,0.1070,-8.166
1,7871.0,11.0,18.0,120.010,0.0591,0.380,0.2730,-8.058
2,14059.0,30.0,60.0,73.000,0.0429,0.210,0.1090,-17.755
3,2471.0,10.0,31.0,69.612,0.0492,0.436,0.0913,-30.246
4,1472617.0,7.0,50.0,150.756,0.0329,0.476,0.1640,-9.491
...,...,...,...,...,...,...,...,...
179935,37474.0,1.0,33.0,128.011,0.0528,0.807,0.0445,-11.361
179936,478.0,8.0,0.0,120.064,0.1320,0.548,0.1220,-11.452
179937,40725.0,1.0,36.0,123.006,0.0532,0.803,0.0618,-6.241
179938,10523.0,25.0,32.0,92.999,0.0376,0.655,0.3370,-7.407


In [25]:
uniform_features_df = sample_df[uniform_features]
uniform_features_df

,energy,valence,acousticness,instrumentalness
0,0.345,0.1560,0.767000,0.82500
1,0.845,0.2950,0.000017,0.72700
2,0.102,0.0382,0.955000,0.78600
3,0.014,0.0886,0.993000,0.90100
4,0.501,0.6600,0.665000,0.00000
...,...,...,...,...
179935,0.709,0.7400,0.002320,0.78600
179936,0.892,0.2850,0.000702,0.00664
179937,0.743,0.1760,0.001200,0.30800
179938,0.756,0.4010,0.009920,0.02800


In [26]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
normal_features_scaled = pd.DataFrame(std_scaler.fit_transform(normal_features_df), columns=normal_features, index=normal_features_df.index)
normal_features_scaled

,artist_followers,album_total_tracks,artist_popularity,tempo,speechiness,danceability,liveness,loudness
0,-0.231849,-0.554068,-0.070729,1.338886,-0.366063,-0.181690,-0.491363,0.397467
1,-0.230750,-0.146350,-1.101348,0.028699,-0.226903,-0.879198,0.508441,0.413109
2,-0.229717,0.628316,0.959889,-1.509792,-0.396407,-1.770751,-0.479317,-0.991296
3,-0.231652,-0.187121,-0.463346,-1.620670,-0.330488,-0.585511,-0.585923,-2.800353
4,0.013836,-0.309437,0.469118,1.034919,-0.501039,-0.375733,-0.148057,0.205569
...,...,...,...,...,...,...,...,...
179935,-0.225807,-0.554068,-0.365192,0.290546,-0.292821,1.360172,-0.867795,-0.065261
179936,-0.231985,-0.268665,-1.984735,0.030466,0.535865,0.001865,-0.401019,-0.078440
179937,-0.225264,-0.554068,-0.217961,0.126748,-0.288636,1.339194,-0.763599,0.676263
179938,-0.230307,0.424457,-0.414269,-0.855287,-0.451862,0.563019,0.893907,0.507392


In [27]:
from sklearn.preprocessing import MinMaxScaler

minmax_scaler = MinMaxScaler()
uniform_features_scaled = pd.DataFrame(minmax_scaler.fit_transform(uniform_features_df), columns=uniform_features, index=uniform_features_df.index)
uniform_features_scaled

,energy,valence,acousticness,instrumentalness
0,0.345,0.1560,0.770080,0.82500
1,0.845,0.2950,0.000017,0.72700
2,0.102,0.0382,0.958835,0.78600
3,0.014,0.0886,0.996988,0.90100
4,0.501,0.6600,0.667671,0.00000
...,...,...,...,...
179935,0.709,0.7400,0.002329,0.78600
179936,0.892,0.2850,0.000705,0.00664
179937,0.743,0.1760,0.001205,0.30800
179938,0.756,0.4010,0.009960,0.02800


In [28]:
print(sample_df.columns)

Index(['artist_followers', 'genres', 'album_total_tracks', 'artist_popularity',
       'explicit', 'tempo', 'chart', 'album_release_date', 'energy', 'key',
       'popularity', 'available_markets', 'mode', 'time_signature',
       'speechiness', 'danceability', 'valence', 'acousticness', 'liveness',
       'instrumentalness', 'loudness'],
      dtype='object')


In [33]:
unchanged_features = [
    "explicit",
    "chart",
    "album_release_date",
    "popularity",
    "mode"
]

In [34]:
final_transformed_df = pd.concat(
    [
        transformed_genres,
        transformed_av_markets,
        key_time_encoded,
        normal_features_scaled,
        uniform_features_scaled,
        sample_df[unchanged_features],
    ],
    axis=1,
)
print(final_transformed_df.shape)

: 

# Proceed with Feature Expectations

In [30]:
import great_expectations as gx

context = gx.get_context(project_root_dir = "services")
context

{
  "anonymous_usage_statistics": {
    "explicit_url": false,
    "usage_statistics_url": "https://stats.greatexpectations.io/great_expectations/v1/usage_statistics",
    "explicit_id": true,
    "enabled": true,
    "data_context_id": "5f79ca94-91e9-4106-82ce-e07c02b3bae2"
  },
  "checkpoint_store_name": "checkpoint_store",
  "config_variables_file_path": "uncommitted/config_variables.yml",
  "config_version": 3.0,
  "data_docs_sites": {
    "local_site": {
      "class_name": "SiteBuilder",
      "show_how_to_buttons": true,
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "uncommitted/data_docs/local_site/"
      },
      "site_index_builder": {
        "class_name": "DefaultSiteIndexBuilder"
      }
    }
  },
  "datasources": {},
  "evaluation_parameter_store_name": "evaluation_parameter_store",
  "expectations_store_name": "expectations_store",
  "fluent_datasources": {},
  "include_rendered_content": {
    "globally": false

In [31]:
ds = context.sources.add_or_update_pandas(name = "pandas_datasource")
ds

PandasDatasource(type='pandas', name='pandas_datasource', id=None, assets=[])